In [80]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import random
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from category_encoders import OneHotEncoder, OrdinalEncoder
import pickle

import sqlite3

In [2]:
# DB_FILENAME = 'project3.db'
# DB_FILEPATH = os.path.join(os.getcwd(), DB_FILENAME)
DB_FILEPATH = 'd:\\4my\\dev\\python\\codestates\\Section3\\project3\\project3.db'
conn = sqlite3.connect(DB_FILEPATH)
cur = conn.cursor()

In [3]:
df = pd.read_sql('SELECT * FROM game_info;', conn)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127889 entries, 0 to 127888
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   appid            127889 non-null  object 
 1   name             127889 non-null  object 
 2   developer        127889 non-null  object 
 3   publisher        127889 non-null  object 
 4   score_rank       127889 non-null  object 
 5   positive         127889 non-null  int64  
 6   negative         127889 non-null  int64  
 7   userscore        127889 non-null  int64  
 8   owners           127889 non-null  object 
 9   average_forever  127889 non-null  int64  
 10  average_2weeks   127889 non-null  int64  
 11  median_forever   127889 non-null  int64  
 12  median_2weeks    127889 non-null  int64  
 13  price            127889 non-null  float64
 14  initialprice     127889 non-null  float64
 15  discount         127889 non-null  object 
 16  ccu              127889 non-null  int6

In [5]:
df.drop(columns=['score_rank'],inplace=True);

In [6]:
df['userscore'].value_counts()

0      127246
90         46
100        41
89         24
80         21
        ...  
52          1
6           1
16          1
54          1
29          1
Name: userscore, Length: 84, dtype: int64

In [7]:
df['owners'].value_counts()

0 .. 20,000                   110667
20,000 .. 50,000                7201
50,000 .. 100,000               3571
100,000 .. 200,000              2449
200,000 .. 500,000              2060
500,000 .. 1,000,000             924
1,000,000 .. 2,000,000           528
2,000,000 .. 5,000,000           335
5,000,000 .. 10,000,000           90
10,000,000 .. 20,000,000          37
20,000,000 .. 50,000,000          21
50,000,000 .. 100,000,000          5
100,000,000 .. 200,000,000         1
Name: owners, dtype: int64

In [8]:
df['discount'] = df['discount'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127889 entries, 0 to 127888
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   appid            127889 non-null  object 
 1   name             127889 non-null  object 
 2   developer        127889 non-null  object 
 3   publisher        127889 non-null  object 
 4   positive         127889 non-null  int64  
 5   negative         127889 non-null  int64  
 6   userscore        127889 non-null  int64  
 7   owners           127889 non-null  object 
 8   average_forever  127889 non-null  int64  
 9   average_2weeks   127889 non-null  int64  
 10  median_forever   127889 non-null  int64  
 11  median_2weeks    127889 non-null  int64  
 12  price            127889 non-null  float64
 13  initialprice     127889 non-null  float64
 14  discount         127889 non-null  int32  
 15  ccu              127889 non-null  int64  
 16  languages        127889 non-null  obje

In [9]:
df['publisher'] = df['publisher'].replace('',np.nan, regex=True)
df['developer'] = df['developer'].replace('',np.nan, regex=True)
drop_index = df[(df['publisher'].isna()) & df['developer'].isna()].index
df.drop(index=drop_index, inplace=True)
df['publisher'] = np.where(df['publisher'].isna()==True, df['developer'], df['publisher'])
df['developer'] = np.where(df['developer'].isna()==True, df['publisher'], df['developer'])

In [10]:
df['tags']= df['tags'].str.count(',')+1
df['tags'].astype(int)

0         20
1         20
3         20
4         20
5         20
          ..
127884     1
127885    20
127886     1
127887     1
127888     1
Name: tags, Length: 114792, dtype: int32

In [11]:
df['languages']= df['languages'].str.count(',')+1
df['languages'].astype(int)

0         8
1         8
3         3
4         9
5         1
         ..
127884    1
127885    1
127886    1
127887    1
127888    1
Name: languages, Length: 114792, dtype: int32

In [12]:
df['genre']= df['genre'].str.count(',')+1
df['genre'].astype(int)

0         1
1         1
3         3
4         4
5         4
         ..
127884    1
127885    4
127886    3
127887    5
127888    3
Name: genre, Length: 114792, dtype: int32

In [13]:
target = 'owners'
features = df.drop(columns=['owners','name','developer','publisher']).columns
train, test = train_test_split(df, train_size=0.9, test_size=0.1)

In [14]:
X_train = train[features]
X_test = test[features]
y_train = train[target]
y_test = test[target]

In [15]:
X_train

,appid,positive,negative,userscore,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,languages,genre,tags
58122,2006300,0,0,0,0,0,0,0,1.99,1.99,0,0,1,1,1
26695,1373560,1,2,0,0,0,0,0,9.99,9.99,0,0,1,2,15
4745,1067220,0,0,0,0,0,0,0,0.00,0.00,0,0,1,4,1
8984,1123270,0,0,0,0,0,0,0,2.99,2.99,0,0,2,2,1
39728,1576720,20,3,0,0,0,0,0,9.99,9.99,0,1,1,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99936,657420,2,0,0,0,0,0,0,9.99,9.99,0,0,5,1,1
106211,735530,12,5,0,0,0,0,0,0.00,0.00,0,0,1,3,9
48063,1742340,12,3,0,0,0,0,0,9.99,9.99,0,0,1,5,17
98707,644130,9,33,0,0,0,0,0,0.00,0.00,0,0,4,2,14


In [16]:
enc = OrdinalEncoder()
# enc.fit(X.)

In [17]:
model = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=7, min_samples_split=10)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=7, min_samples_split=10)

In [18]:
y_pred = model.predict(X_test)

In [19]:
model.score(X_test,y_test)

0.8749128919860627

In [20]:
model.predict_proba(X_test)

array([[3.13739011e-01, 1.55011937e-03, 0.00000000e+00, ...,
        1.98110226e-01, 0.00000000e+00, 5.24470674e-03],
       [9.49098835e-01, 5.15405345e-05, 0.00000000e+00, ...,
        9.38246442e-03, 0.00000000e+00, 2.29160637e-04],
       [9.77439257e-01, 2.73197574e-05, 0.00000000e+00, ...,
        3.01568463e-03, 0.00000000e+00, 6.84811563e-05],
       ...,
       [9.91281024e-01, 6.67033385e-06, 0.00000000e+00, ...,
        1.30822912e-03, 0.00000000e+00, 2.11058865e-05],
       [9.61177870e-01, 3.12456546e-05, 0.00000000e+00, ...,
        5.45483397e-03, 0.00000000e+00, 1.30486219e-04],
       [8.68736636e-01, 1.86489289e-04, 0.00000000e+00, ...,
        2.59136360e-02, 0.00000000e+00, 8.55740824e-04]])

In [21]:
df_copy = df.copy()

df_copy['reviews'] = df_copy.apply(lambda x : x['positive'] + x['negative'], axis=1)
df_copy['oddsR'] = df_copy.apply(lambda x : (x['positive']+1)/(x['negative']+1), axis=1)

In [22]:
def sample_mean_std(count, min, max):
  temp = []
  for i in range(count):
    temp.append(random.randint(min,max))
  mean_ = int(np.mean(temp))
  std_ = int(np.std(temp))
  higher = mean_+1.96*std_
  lower = mean_-1.96*std_
  if lower < 0:
    lower = 0
  return mean_

In [23]:
df_copy['owners_max'] = df_copy['owners'].str.replace(',','').str.split(' .. ').str[1]
df_copy['owners_min'] = df_copy['owners'].str.replace(',','').str.split(' .. ').str[0]
df_copy[['owners_min','owners_max']]=df_copy[['owners_min','owners_max']].astype(int)

In [24]:
df_copy['sale'] = df_copy.apply(lambda x: sample_mean_std(17,x['owners_min'],x['owners_max']),axis=1)
df_copy.drop(columns=['owners_max','owners_min'],inplace=True)

In [25]:
df_copy

,appid,name,developer,publisher,positive,negative,userscore,owners,average_forever,average_2weeks,...,price,initialprice,discount,ccu,languages,genre,tags,reviews,oddsR,sale
0,10,Counter-Strike,Valve,Valve,199209,5149,0,"10,000,000 .. 20,000,000",10388,891,...,9.99,9.99,0,12423,8,1,20,204358,38.681553,15120228
1,100,Counter-Strike: Condition Zero,Valve,Valve,13442,1535,0,"10,000,000 .. 20,000,000",598,9748,...,9.99,9.99,0,69,8,1,20,14977,8.751953,15866922
3,1000000,ASCENXION,IndigoBlue Game Studio,PsychoFlux Entertainment,27,5,0,"0 .. 20,000",0,0,...,9.99,9.99,0,0,3,3,20,32,4.666667,8981
4,1000010,Crown Trick,NEXT Studios,"Team17, NEXT Studios",3930,613,0,"200,000 .. 500,000",483,0,...,19.99,19.99,0,39,9,4,20,4543,6.402280,382498
5,1000030,"Cook, Serve, Delicious! 3?!",Vertigo Gaming Inc.,Vertigo Gaming Inc.,1521,110,0,"100,000 .. 200,000",126,0,...,7.99,19.99,60,66,1,4,20,1631,13.711712,153565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127884,927,GTI Racing Trailer,Techland,Topware Interactive,0,0,0,"0 .. 20,000",0,0,...,0.00,0.00,0,0,1,1,1,0,1.000000,10620
127885,926990,WolfQuest: Anniversary Edition,eduweb,eduweb,4400,148,0,"100,000 .. 200,000",48223,0,...,19.99,19.99,0,212,1,4,20,4548,29.536913,147450
127886,926980,Fantasy Grounds - Quests of Doom 4: God of Ore...,"SmiteWorks USA, LLC","SmiteWorks USA, LLC",0,0,0,"0 .. 20,000",0,0,...,6.39,7.99,20,0,1,3,1,0,1.000000,10072
127887,926970,Hypatia - Starter Pack,TimefireVR Inc,TimefireVR Inc,0,0,0,"0 .. 20,000",0,0,...,9.99,9.99,0,0,1,5,1,0,1.000000,8552


In [73]:
target = 'sale'
features = ['reviews','oddsR','positive']
# features = df_copy.drop(columns=['owners','name','developer','publisher','positive','negative']).columns
train1, test1 = train_test_split(df_copy, train_size=0.9, test_size=0.1)

In [74]:
X_train = train1[features]
X_test = test1[features]
y_train = train1[target]
y_test = test1[target]

In [75]:
X_train

,reviews,oddsR,positive
80933,126,3.740741,100
43213,13,6.500000,12
82029,0,1.000000,0
101956,600,4.522936,492
84647,144,3.171429,110
...,...,...,...
12300,0,1.000000,0
125552,22,3.800000,18
116262,51,7.833333,46
68482,8,9.000000,8


In [76]:
model = RandomForestRegressor(n_estimators=400, criterion='squared_error', max_depth=10, min_samples_leaf=10, random_state=99)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(model.score(X_test,y_test))

0.7074188955395485


In [82]:
with open ('model.pkl','wb') as pickle_file:
    pickle.dump(model, pickle_file) #reviews, odds, positive

In [89]:
# PER 구하기
df_copy[df_copy['name']=='Crown Trick']

,appid,name,developer,publisher,positive,negative,userscore,owners,average_forever,average_2weeks,...,price,initialprice,discount,ccu,languages,genre,tags,reviews,oddsR,sale
4,1000010,Crown Trick,NEXT Studios,"Team17, NEXT Studios",3930,613,0,"200,000 .. 500,000",483,0,...,19.99,19.99,0,39,9,4,20,4543,6.40228,382498


In [78]:

group_df = df_copy.groupby(by=['owners']).mean()['reviews']
owners_template = pd.DataFrame(group_df)
owners_template['per'] = [int(10000/owners_template.iloc[0]),int(1500000/owners_template.iloc[1]),int(15000000/owners_template.iloc[2]),int(150000/owners_template.iloc[3]),int(150000000/owners_template.iloc[4]),int(3500000/owners_template.iloc[5]),int(35000/owners_template.iloc[6]),int(35000000/owners_template.iloc[7]),int(350000/owners_template.iloc[8]),int(7500000/owners_template.iloc[9]),int(75000/owners_template.iloc[10]),int(75000000/owners_template.iloc[11]),int(750000/owners_template.iloc[12])]

In [79]:
owners_template

,reviews,per
owners,,
"0 .. 20,000",2.544509e+02,39
"1,000,000 .. 2,000,000",1.960993e+04,76
"10,000,000 .. 20,000,000",2.463515e+05,60
"100,000 .. 200,000",1.227977e+03,122
"100,000,000 .. 200,000,000",1.795138e+06,83
"2,000,000 .. 5,000,000",5.220278e+04,67
"20,000 .. 50,000",2.175383e+02,160
"20,000,000 .. 50,000,000",5.485320e+05,63
"200,000 .. 500,000",3.032343e+03,115


In [87]:
arr = np.array([[10357, 8.4, 7042]])
model.predict(arr)

c:\Users\jsm88\anaconda3\envs\project\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


array([976604.52368237])